Creating first MVP product

## Import Libraries

Installations for extra packages: 
- !pip install SpeechRecognition
- !pip install PocketSphinx
- !brew install portaudio
- !pip install pyaudio

## Data Sources:

- NYT daily: http://podbay.fm/show/1200361736/e/1527587699?autostart=1 ; 
- NYT daily: https://itunes.apple.com/us/podcast/the-daily/id1200361736?mt=2&ign-mpt=uo%3D4
- Open Audio Source: http://www.openslr.org/resources.php
- Economist: https://www.economist.com/audio-edition/2018-06-02
- NPR: https://www.npr.org/programs/morning-edition/


## Speech Recognition

- Real python: https://realpython.com/python-speech-recognition/#working-with-audio-files
- Speech Recognition: https://github.com/Uberi/speech_recognition#readme
- Speech Recognition: https://pypi.org/project/SpeechRecognition/3.2.0/
- Pocket Sphinx: https://pypi.org/project/pocketsphinx/

## Natural Language Processing

- Insight: https://blog.insightdatascience.com/how-to-solve-90-of-nlp-problems-a-step-by-step-guide-fda605278e4e
- Summary: https://machinelearningmastery.com/gentle-introduction-text-summarization/

In [1]:
import speech_recognition as sr
import time as time

In [2]:
!ls Data/EC_06022018

78 078 Science and technology - Evolution.wav
79 079 Science and technology - Conservation.wav
80 080 Science and technology - The sunk-cost fallacy.wav
81 081 Science and technology - AIDS.wav
82 082 Science and technology - Space flight.wav


In [3]:
## Intitialize a recognizer
r_live = sr.Recognizer()
r_rec  = sr.Recognizer()

In [9]:
# obtain audio from the microphone
with sr.Microphone() as source:
    print("Say something!")
    audio = r_live.listen(source)

Say something!


In [7]:
test = sr.AudioFile('Data/EC_06022018/82 082 Science and technology - Space flight.wav')
with test as source:
    test_au = r_rec.record(source, duration=30)

In [19]:
start_time = time.time()
print("Sphinx:", r_rec.recognize_sphinx(test_au, language='en-US', keyword_entries=None, grammar=None, show_all=False))
print("--- %s seconds ---" % (time.time() - start_time))

Sphinx: half and half science and technology space flight munching into all bit of rockets that's divine was itself may seem take off to a tumult of omen for the motion sensor large into space typically the payload represented me about five percent of americans over rock it was it leaves the lord for the rockets botha's account with some of the rest but the bulk of it consists of the propellant
--- 8.8052659034729 seconds ---


In [10]:
start_time = time.time()
print("Sphinx:", r_live.recognize_sphinx(audio, language='en-US', keyword_entries=None, grammar=None, show_all=False))
print("--- %s seconds ---" % (time.time() - start_time))

Sphinx: it's not you can hear me these reporters must be causing warren's fenced off the foodstuffs involving the fifth floor
--- 7.880022048950195 seconds ---


In [12]:
audio